In [3]:
from Bio import SeqIO

In [34]:
genome = 'sequence.full.w.gi.ape'
seq = SeqIO.read(genome, 'gb')

Sequence can also be downloaded by the following:
Accession numbers are the preferred method of fetching.

In [ ]:
from Bio import Entrez

In [2]:
accession = 'NC_000913.3'
results = Entrez.efetch(db='nucleotide', id=accession, rettype='gbwithparts')
record = results.read()

/usr/local/lib/python3.9/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(


## Getting our gene from genome
We are looking to interact with the `mak` gene
Manually looking at `ape` file shows that it occurs under index `1647`

In [27]:
seq.features[1647].qualifiers.keys()

odict_keys(['gene', 'locus_tag', 'gene_synonym', 'EC_number', 'codon_start', 'transl_table', 'product', 'protein_id', 'db_xref', 'translation'])

Gene titles seem to be located at `seq.features[i].qualifiers.gene[x]`
Therefore we need a function to iterate through `seq` and filter through gene names

In [52]:
# Iterate through `seq.features`
def search(seq, term):
    for feature in seq.features:
        if 'gene' in feature.qualifiers.keys() and term in feature.qualifiers['gene']:
            return feature

In [51]:
mak = search(seq, 'mak')

## Getting metadata on gene

Using db value 'ECOCYC:EG11288', Entrez can be used to search

In [56]:
Entrez.email = 'poor.rican@pm.me'
results = Entrez.esearch(db='gene', term='EG11288')
record = Entrez.read(results)
id = record['IdList'][0]

In [59]:
id

'949086'

In [132]:
results = Entrez.efetch(db='gene', id='949086', rettype='gbwithparts', retmode='text')
record = results.read()
record

'\n1. mak\nfructokinase [Escherichia coli str. K-12 substr. MG1655]\nOther Aliases: b0394, ECK0389, yajF\nOther Designations: fructokinase\nAnnotation:  NC_000913.3 (410144..411052)\nID: 949086\n\n'

`id` is required argument otherwise `HTTP 400`

In [87]:
results = Entrez.efetch(db='gene', id='949086', rettype="gb", retmode='text')

In [88]:
text = results.read()

In [89]:
text

'\n1. mak\nfructokinase [Escherichia coli str. K-12 substr. MG1655]\nOther Aliases: b0394, ECK0389, yajF\nOther Designations: fructokinase\nAnnotation:  NC_000913.3 (410144..411052)\nID: 949086\n\n'

In [90]:
print(text)


1. mak
fructokinase [Escherichia coli str. K-12 substr. MG1655]
Other Aliases: b0394, ECK0389, yajF
Other Designations: fructokinase
Annotation:  NC_000913.3 (410144..411052)
ID: 949086




Only basic info is returned through gene database. Let's see what the nucleotide database returns.

## Getting protein data
How to get protein data from gene?
`ECK0389` seemed to return a value.
### Problem
Therefore, we need a method that takes list of gene names/id's and searches for proteins.

In [8]:
results = Entrez.esearch(db='protein', term='ECK0389')
record = Entrez.read(results)

In [145]:
print(record)

{'Count': '9', 'RetMax': '9', 'RetStart': '0', 'IdList': ['90111128', '87081733', '1894859918', '1775789164', '359331155', '85674535', '315135077', '802133986', '682117968'], 'TranslationSet': [], 'TranslationStack': [{'Term': 'ECK0389[All Fields]', 'Field': 'All Fields', 'Count': '9', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': 'ECK0389[All Fields]'}


We then need a function that takes a list of id's and gets files

In [19]:
def multifetch(db, ids, rettype):
    returned = []
    for id in ids:
        with Entrez.efetch(db=db, id=id, rettype=rettype) as handle:
            returned.append(SeqIO.read(handle, 'gb'))
    return returned

In [147]:
record['IdList']

['90111128', '87081733', '1894859918', '1775789164', '359331155', '85674535', '315135077', '802133986', '682117968']

In [20]:
proteins = multifetch('protein', record['IdList'], 'gp')

Since multiple proteins were returned from `esearch`, each `id` must be verified.

No detailed information is passed except for special sites which can be annotated by [GO](http://geneontology.org/docs/introduction-to-go-resource/)
I don't know if this can be integrated with COBRApy, et al.

# Get list of organism genomes (function)

In [55]:
def org_genome_list(org):
    return Entrez.esearch(db='genome', term='%s[organism]' % (org))

In [56]:
raw = org_genome_list('e coli')

In [58]:
list = Entrez.read(raw)

In [59]:
list

{'Count': '1', 'RetMax': '1', 'RetStart': '0', 'IdList': ['167'], 'TranslationSet': [{'From': 'e coli[organism]', 'To': '"Escherichia coli"[Organism]'}], 'TranslationStack': [{'Term': '"Escherichia coli"[Organism]', 'Field': 'Organism', 'Count': '1', 'Explode': 'Y'}, 'GROUP'], 'QueryTranslation': '"Escherichia coli"[Organism]'}

Search came up w/ only one `id`: 167

In [64]:
with Entrez.esearch(db='genome', term='167[id]') as handle:
    thing = Entrez.read(handle)
thing

{'Count': '314', 'RetMax': '20', 'RetStart': '0', 'IdList': ['117023', '116345', '116331', '115705', '115223', '115200', '115197', '115190', '115185', '115163', '115162', '114126', '112851', '111561', '108858', '108301', '108300', '108299', '107518', '107514'], 'TranslationSet': [], 'TranslationStack': [{'Term': '167[All Fields]', 'Field': 'All Fields', 'Count': '314', 'Explode': 'N'}, 'GROUP'], 'QueryTranslation': '167[All Fields]', 'ErrorList': {'PhraseNotFound': [], 'FieldNotFound': ['id']}}

In [67]:
with Entrez.esearch(db='nuccore', term='e coli', retmode='text') as handle:
    thing = Entrez.read(handle)
    print(thing)

{'Count': '19557228', 'RetMax': '20', 'RetStart': '0', 'IdList': ['2305351642', '2305351641', '2305351640', '2305351639', '2305351638', '2305351637', '2305351636', '2305351635', '2305351634', '2305351633', '2305351632', '2305344037', '2305338941', '2305338940', '2305338938', '2305338937', '2305338936', '2305338935', '2305338934', '2305338931'], 'TranslationSet': [{'From': 'e coli', 'To': '"Escherichia coli"[Organism] OR e coli[All Fields]'}], 'TranslationStack': [{'Term': '"Escherichia coli"[Organism]', 'Field': 'Organism', 'Count': '11096220', 'Explode': 'Y'}, {'Term': 'e coli[All Fields]', 'Field': 'All Fields', 'Count': '9829475', 'Explode': 'N'}, 'OR', 'GROUP'], 'QueryTranslation': '"Escherichia coli"[Organism] OR e coli[All Fields]'}


It looks like a linking `id` is given to link to main organism page.
In that case, it looks like organism taxonomies should be hardcoded. The idea is to group genomes of all organisms by taxonomy.

Looks like [FTP is available](https://ftp.ncbi.nlm.nih.gov/genomes/genbank/) and should contain representative genomes.
But how do we get representative genomes by search term?
This must be put on hold until main features are setup.

In [93]:
from xml.etree import ElementTree

In [160]:
def get_genome_list(organism):
    # TODO: have more expansive query
    query = '"%s"[Organism] AND (Refseq[Filter] AND "bioproject nuccore"[Filter] AND "scope monoisolate"[Filter])'\
            % organism
    with Entrez.esearch(db='bioproject', term=query, retmode='text') as handle:
        records = Entrez.read(handle)

    accessions = []
    for id in records['IdList']:
        with Entrez.efetch(db='bioproject', id=id) as handle:
            root = ElementTree.parse(handle).getroot()
            accession = root[0][0][0][0].attrib['accession']
            accessions.append(accession)

    ids = []
    for accession in accessions:
        with Entrez.esearch(db='nuccore', term=accession) as handle:
            ids.extend(Entrez.read(handle)['IdList'])

    genomes = []
    for id in ids:
        print("Trying %s" % id)
        with Entrez.efetch(db='nuccore', id=id, rettype='gb', retmode='text') as handle:
            genomes.append(SeqIO.read(handle, 'gb'))
    # exclude any sequence that has "plasmid" in title
    return [i for i in genomes if 'plasmid' not in i.description]


In [161]:
genomes = get_genome_list('e coli')

Trying 10955266
Trying 10955262
Trying 556503834
